# Bucketing

We already saw that using a prepartitioned DataFrame for joins and grouped aggregations can accelerate execution time, but so far the prepartitioning had to be performed every time data is loaded from disk. It would be really nice, if there was some way to store prepartitioned data, such that Spark understands which columns were used to create the partitions.

This is where bucketing comes into play, which is a special way to store data, such that Spark actually understands the partitioning schema.

### Adjust Spark config
Again, we need to disable automatic broadcast joins and make sure that bucketing is enabled.

In [ ]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
spark.conf.set("spark.sql.sources.bucketing.enabled", True)

# 1 Load Data

First we load the weather data, which consists of the measurement data and some station metadata.

In [ ]:
storageLocation = "s3://dimajix-training/data/weather"

## 1.1 Load Measurements

Measurements are stored in multiple directories (one per year). But we will limit ourselves to a single year in the analysis to improve readability of execution plans.

In [ ]:
from pyspark.sql.functions import *
from functools import reduce

# Read in all years, store them in an Python array
raw_weather_per_year = [spark.read.text(storageLocation + "/" + str(i)).withColumn("year", lit(i)) for i in range(2003,2015)]

# Union all years together
raw_weather = reduce(lambda l,r: l.union(r), raw_weather_per_year)                        

Use a single year to keep execution plans small

In [ ]:
raw_weather = spark.read.text(storageLocation + "/2003").withColumn("year", lit(2003))

### Extract Measurements

Measurements were stored in a proprietary text based format, with some values at fixed positions. We need to extract these values with a simple `SELECT` statement.

In [ ]:
weather = raw_weather.select(
    col("year"),
    substring(col("value"),5,6).alias("usaf"),
    substring(col("value"),11,5).alias("wban"),
    substring(col("value"),16,8).alias("date"),
    substring(col("value"),24,4).alias("time"),
    substring(col("value"),42,5).alias("report_type"),
    substring(col("value"),61,3).alias("wind_direction"),
    substring(col("value"),64,1).alias("wind_direction_qual"),
    substring(col("value"),65,1).alias("wind_observation"),
    (substring(col("value"),66,4).cast("float") / lit(10.0)).alias("wind_speed"),
    substring(col("value"),70,1).alias("wind_speed_qual"),
    (substring(col("value"),88,5).cast("float") / lit(10.0)).alias("air_temperature"),
    substring(col("value"),93,1).alias("air_temperature_qual")
)

## 1.2 Load Station Metadata

We also need to load the weather station meta data containing information about the geo location, country etc of individual weather stations.

In [ ]:
stations = spark.read \
    .option("header", True) \
    .csv(storageLocation + "/isd-history")

# 2 Bucketing Data

Now we want to create a so called *bucketed table* of the weather measurements data. Bucketing is only possible within Hive, because additional meta data about the bucketing is required. That meta data is not stored on HDFS but persisted in the Hive metastore instead.

## 2.1 Create Hive Table

A bucketed Hive table can easily be created from within Spark by using the `bucketBy` and optionally `sortBy` method of the `DataFrameWriter` class.

In [ ]:
## YOUR CODE HERE

## 2.2 Inspect Table

We can inspect the Hive table, which unverils that both bucketing columns and sorting columns are present in the Hive table.

In [ ]:
## YOUR CODE HERE

### `CREATE TABLE` statement

We could also have used Hive SQL to create the table. Let's retrieve the statement via SQL `SHOW CREATE TABLE`

In [ ]:
## YOUR CODE HERE

### Inspect Files
Of couse there also need to be some files in HDFS now. These are stored in the directory `/user/hive/warehouse/weather_buckets`

In [ ]:
## YOUR CODE HERE

# 3 Bucketing & Joins

Now we can perform the same join again, but this time against the bucketed version of the weather table.

## 3.1 Normal Join

To see the effect of bucketing, we first perform a traditional join to see the execution plan as a reference.

In [ ]:
result = weather.join(stations, (weather["usaf"] == stations["usaf"]) & (weather["wban"] == stations["wban"]))
result.explain()

## 3.2 Bucketed Join

Now we want to replace the original `weather` DataFrame by a bucketed version. This means that first we have to create a bucketed version in HDFS. This is only possible by creating a Hive table, since this is the only way to persist the bucketing information as table properties.

In [ ]:
weather_hive = ## YOUR CODE HERE
result = ## YOUR CODE HERE
result.explain()

#### Remarks
The execution plan now looks differently than before.
* The station meta data is still shuffled (we didn't bucketize it)
* The weather data does not require a shuffle any more, the join can be executed almost directly (A sort will still be performed, maybe a bug?)

### Bucketing Strategie

The following attributes have to match
* bucketing columns
* number of buckets = number of partitions

# 4 Bucketing & Aggregation

Similar to `JOIN` operations, grouped aggregations (`GROUP BY`) also require a shuffle operation. Again this can be avoided if a Hive table is used that is already bucketed according to the grouping columns.

## 4.1 Normal Aggregation

First let us analyze the execution plan of a normal grouped aggregation operation without a bucketed table. This will result in an execution plan containing a shuffle operation

In [ ]:
result = weather.groupBy(weather["usaf"], weather["wban"]).agg(
        min(when(weather.air_temperature_qual == lit(1), weather.air_temperature)).alias('min_temp'),
        max(when(weather.air_temperature_qual == lit(1), weather.air_temperature)).alias('max_temp'),
)
result.explain()

### Remarks
As expected the execution plan has three steps related to the grouped aggregation:
1. Partial aggregate (`HashAggregate`)
2. Shuffle operation (`Exchange hashpartitioning`)
3. Final aggregate (`HashAggregate`)

## 4.2 Bucketed Aggregation

Now let's perform the same operation, but this time using the bucketed Hive table.

In [ ]:
result = ## YOUR CODE HERE
result.explain()

### Remarks
As we hoped for, Spark will not perform a shuffle operation any more, since the data is already partitioned as needed. The execution plan now only contains two steps for implementing the grouped aggregation
1. Partial aggregate (`HashAggregate`)
2. Final aggregate (`HashAggregate`)

# 5 Bucketing & Filtering

Unfortunately Spark does not use bucketing information for filtering yet. Let's prove that by a simple example.

## 5.1 Filter without bucketing

Let read in the raw data and add a filter operation that refers to the bucketing columns.

In [ ]:
result = ## YOUR CODE HERE
result.explain()

In [ ]:
## YOUR CODE HERE

## 5.2 Filter with bucketing

Now let's try the same example, but this time we use the bucketed Hive table instead of the raw data.

In [ ]:
result = ## YOUR CODE HERE
result.explain()

In [ ]:
## YOUR CODE HERE

### Remarks
The execution plan contains *`PushedFilters`*, but the Spark web ui will reveil, that these filters are only pushed down to the parquet reader and Spark still reads all files.